In [1]:
import pandas as pd
import numpy as np
from haversine import haversine
import datetime
import os
import json
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))

In [2]:
SOTF_list=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_SOTF_Faith_20190311/BL_3.29 and 4.26 Zips Codes Updated 3.11.19 (1).xlsx")
SOTF_list.sheet_names

['BL_3.29 Zips_2 Stores ', 'BL_4.26 Zips_5 Stores']

In [3]:
list_1=SOTF_list.parse('BL_3.29 Zips_2 Stores ',dtype=str)
list_2=SOTF_list.parse('BL_4.26 Zips_5 Stores',dtype=str)

SOTF_store_list=list_1.append(list_2)
SOTF_store_list.shape

(8, 9)

In [4]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",
                             dtype=str,usecols=['location_id','zip','revenue_flag'])
store_level_zips=store_level_zips[store_level_zips['location_id'].isin(SOTF_store_list['Store ID'].tolist())]
store_level_zips=store_level_zips[store_level_zips['revenue_flag'].isin(['P','S'])]
store_level_zips['location_id'].unique().tolist()

['1243', '4555']

In [5]:
store_list=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20190301-134800-131.txt",dtype=str,sep="|")
store_list=store_list[store_list['location_id'].isin(SOTF_store_list['Store ID'])]
# All new stores for the others
store_list=store_list[['location_id','zip_cd','latitude_meas','longitude_meas']].rename(columns={"zip_cd":"Store_Zip","latitude_meas":"Latitude_meas","longitude_meas":"Longitude_meas"})
store_list['Latitude_meas']=store_list['Latitude_meas'].astype(float)
store_list['Longitude_meas']=store_list['Longitude_meas'].astype(float)
store_list['Store_Zip']=store_list['Store_Zip'].apply(lambda x: x.split("-")[0])

In [6]:
store_level_zips=pd.merge(store_level_zips,store_list,on="location_id",how="left")
store_level_zips=store_level_zips.rename(columns={"zip":"Zip"})
store_level_zips.head(4)

,Zip,revenue_flag,location_id,Store_Zip,Latitude_meas,Longitude_meas
0,47025,P,1243,47001,39.075299,-84.894435
1,47001,P,1243,47001,39.075299,-84.894435
2,47018,P,1243,47001,39.075299,-84.894435
3,47032,S,1243,47001,39.075299,-84.894435


In [7]:
store_level_zips=store_level_zips.reset_index()
del store_level_zips['index']
store_level_zips['Distance (mile)']=np.nan

for i in range(len(store_level_zips)):
    zip_center=zip_centers[store_level_zips['Zip'][i]]
    store_center=[store_level_zips['Latitude_meas'][i],store_level_zips['Longitude_meas'][i]]
    store_level_zips['Distance (mile)'][i]=haversine(zip_center,store_center,miles=True)
    
store_level_zips=store_level_zips[(store_level_zips['revenue_flag']=="P") | (store_level_zips['Distance (mile)']<=20)]
store_level_zips=store_level_zips.rename(columns={"location_id":"Store ID","revenue_flag":"Revenue_flag"})
store_level_zips.head(2)

df_1_old_stores=store_level_zips.copy()

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
new_SOTF_store_list=SOTF_store_list[~SOTF_store_list['Store ID'].isin(store_level_zips['Store ID'])]
print(new_SOTF_store_list.shape)
new_SOTF_store_list

(6, 9)


,Store ID,Market,G.O.,Longitude_meas,Latitude_meas,Revenue_flag,Zip,Store_Zip,Distance (mile)
0,5390,"NILES, OH",2019-03-29 00:00:00,nan,nan,nan,nan,nan,nan
1,4685,"LONGMONT, CO",2019-03-29 00:00:00,nan,nan,nan,nan,nan,nan
1,4690,"HUNTINGTON, WV",2019-04-26 00:00:00,nan,nan,nan,nan,nan,nan
2,5383,"MOUNT PLEASANT, PA",2019-04-26 00:00:00,nan,nan,nan,nan,nan,nan
3,4674,"CITRUS HEIGHTS, CA",2019-04-26 00:00:00,nan,nan,nan,nan,nan,nan
4,4693,"VANCOUVER, WA",2019-04-26 00:00:00,nan,nan,nan,nan,nan,nan


In [9]:
# Google Store Locations
store_locations={}
store_locations.update({"5390":{"Google_Address":"5555 Youngstown Warren Rd #970, Niles, OH 44446","Google_Coor":[41.2152114,-80.7497981],"Store_Zip":"44446"}})
store_locations.update({"4685":{"Google_Address":"2225 Main St,Longmont, CO 80501","Google_Coor":[40.1982203,-105.1025797],"Store_Zip":"80501"}})
store_locations.update({"4690":{"Google_Address":"Pea Ridge Plaza Shopping Center, Huntington, WV 25705","Google_Coor":[38.4098684,-82.3246742],"Store_Zip":"25705"}})
store_locations.update({"5383":{"Google_Address":"100 Crossroads Plaza, Mt Pleasant, PA 15666","Google_Coor":[40.1252354,-79.5509667],"Store_Zip":"15666"}})
store_locations.update({"4674":{"Google_Address":"8501 Auburn Blvd, Citrus Heights, CA 95610","Google_Coor":[38.7211604,-121.2927663],"Store_Zip":"95610"}})
store_locations.update({"4693":{"Google_Address":"11696 NE 76th St,Vancouver, WA 98662","Google_Coor":[45.6788241,-122.5536851],"Store_Zip":"98662"}})



In [10]:
df_2_all_new_stores=pd.DataFrame()
for store_id in new_SOTF_store_list['Store ID'].tolist():
    i_counter=1
    store_center=store_locations[store_id]['Google_Coor']
    for zip_cd in zip_centers.keys():
        dist=haversine(store_center,zip_centers[zip_cd],miles=True)
        if dist<=10:
            df=pd.DataFrame({"Store ID":store_id,"Longitude_meas":store_center[0],"Latitude_meas":store_center[1],
                             "Revenue_flag":"zips_in_10_miles","Zip":zip_cd,"Store_Zip":store_locations[store_id]['Store_Zip'],
                            "Distance (mile)":dist},index=[i_counter])
            i_counter+=1
            df_2_all_new_stores=df_2_all_new_stores.append(df)
    

In [11]:
df_zips=df_1_old_stores.append(df_2_all_new_stores)

In [12]:
SOTF_list=pd.ExcelFile("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_SOTF_Faith_20190311/BL_3.29 and 4.26 Zips Codes Updated 3.11.19 (1).xlsx")
list_1=SOTF_list.parse('BL_3.29 Zips_2 Stores ',dtype=str)
list_2=SOTF_list.parse('BL_4.26 Zips_5 Stores',dtype=str)

cols=list_1.columns.tolist()

list_1=list_1[['Store ID',"Market","G.O."]]
list_2=list_2[['Store ID',"Market","G.O."]]


list_1=pd.merge(list_1,df_zips,on="Store ID",how="left")
list_2=pd.merge(list_2,df_zips,on="Store ID",how="left")
list_1=list_1[cols]
list_2=list_2[cols]


In [13]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2019_Q1/Planner_Request/Zips_SOTF_Faith_20190311/BL_3.29 and 4.26 Zips Codes Updated 3.11.19_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")

list_1.to_excel(writer,"BL_3.29 Zips_3 Stores",index=False)

list_2.to_excel(writer,'BL_4.26 Zips_5 Stores',index=False)

writer.save()